# Steam Marketplace Analysis

#### Jacob Knispel (Team Lead), Nithin Perumal, Alec Tiefenthal, Matt Buckner

### Table of Contents

* [Executive Summary](http://localhost:8888/notebooks/ReportDraft.ipynb#Executive-Summary)
* [Introduction](http://localhost:8888/notebooks/ReportDraft.ipynb#Introduction)
* [Data Preprocessing](http://localhost:8888/notebooks/ReportDraft.ipynb#Data-Preprocessing)
* [Analysis](http://localhost:8888/notebooks/ReportDraft.ipynb#Analysis)
* [Results](http://localhost:8888/notebooks/ReportDraft.ipynb#Results)
* [Links](http://localhost:8888/notebooks/ReportDraft.ipynb#Links)
* [Disclaimer](http://localhost:8888/notebooks/ReportDraft.ipynb#Dislaimer)

## Executive Summary

In this project, we seek to examine the data provided by Valve Software pertaining to their online digital economy of in-game items in their game Team Fortress 2 in the attempt to locate statistically interesting anomalies or trends that may not be immediately obvious to the outside observer.

## Introduction

## Data Preprocessing

In [ ]:
import requests;                                 # import package used to downlaod data from a web URL
import json;
import pandas as pd;
import seaborn as sns;
import matplotlib.pyplot as plt;                 # import all tools used to examine and plot the data
import matplotlib;
from bokeh.charts import Scatter, show
from bokeh.io import output_notebook
output_notebook()

from pandas.io.json import json_normalize

matplotlib.style.use('ggplot');
%matplotlib inline

The following lines utilize the API calls and key to download and store the data for examination

In [ ]:
with open('API Keys.txt', 'r') as keyfile:                              # This key was requested from Valve
    apiKeys=keyfile.read().split('\n')                                  # The specfic key utilized is included
   
url = 'http://backpack.tf/api/IGetPrices/v4/?key=' + apiKeys[0];        # web address of data stream

data_stream = requests.get(url,stream=True);                            # create an iterable data stream object

rec = data_stream.iter_lines().next().strip()                           # Necessary before reading data
data = json.loads(rec)                                                  # convert the json record to a python dictionary

In [ ]:
def byteify(input):
    if isinstance(input, dict):
        return {byteify(key): byteify(value)
                for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [byteify(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

data2 = byteify(data)

In [ ]:
df = json_normalize(data2['response'])

In [ ]:
#Define values provided by Valve.

raw_usd_value = df['raw_usd_value']
current_time = df['current_time']
success = df['success']
usd_currency_index = df['usd_currency_index']
usd_currency = df['usd_currency']

In [ ]:
#Import the item data

#THIS WILL TAKE >2 MINUTES TO RUN

dfItemFinal = pd.DataFrame(columns = ['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex', 'Currency', 'Value', 'Last_Update', 'Difference'])
df = json_normalize(data2['response'], 'items')
for itemName in df[0]:
    dfName = json_normalize(data2['response']['items'], [itemName])
    if(dfName.size != 2):
        raise Exception('More than 2 items for item' + itemName)
    if(dfName[0][0] != 'prices'):
        raise Exception('First row in %s is not prices' % (itemName))
    if(dfName[0][0] != 'prices'):
        raise Exception('Second row in %s is not defindex' % (itemName))
    defIndex = json_normalize(data2['response']['items'][itemName], ['defindex'])[0][0]
    dfPrices = json_normalize(data2['response']['items'][itemName], 'prices')
    for qualInt in dfPrices[0]:
        dfTrade = json_normalize(data2['response']['items'][itemName]['prices'], qualInt)
        
        for tradeable in dfTrade[0]:
            dfCraft = json_normalize(data2['response']['items'][itemName]['prices'][qualInt], tradeable)
        
            for craftable in dfCraft[0]:
                dfPriceIndex = json_normalize(data2['response']['items'][itemName]['prices'][qualInt][tradeable], craftable)
                
                for priceIndex in dfPriceIndex[0]:
                    dfFinalValues = json_normalize(data2['response']['items'][itemName]['prices'][qualInt][tradeable][craftable][priceIndex])
                    
                    currency = dfFinalValues['currency'][0]
                    difference = dfFinalValues['difference'][0]
                    last_update = dfFinalValues['last_update'][0]
                    value = dfFinalValues['value'][0]
                    dfItemFinal.loc[len(dfItemFinal)]=[itemName, defIndex, qualInt, tradeable, craftable, priceIndex, currency, value, last_update, difference]

In [ ]:
#Store the item data in an csv file for formatting and examinination

dfItemFinal.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex'])
#dfItemFinal = dfItemFinal.drop('Unnamed: 0', axis=1)
dfItemFinal.sort_index(inplace = True)
dfItemFinal.to_csv('itemOut.csv')

Once the data is stored in a .csv file, the following line prepares it for examination.
If a set of data has already been prepared, one may skip directly to this point to begin working with the dataset.

In [ ]:
#Move to our primary dataframe and access our item data

dfNew = pd.read_csv('itemOut.csv')
#dfNew = dfNew.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex'])
dfNew = dfNew.drop('Unnamed: 0', axis=1)
dfNew.sort_index(inplace = True)

## Analysis

## Results

## Links

[API keys.txt](https://github.com/NithinPerumal/SteamMarketAnalysis/blob/master/IPython%20Notebook/API%20Keys.txt "API keys.txt")

[Sample CSV dataset](https://github.com/NithinPerumal/SteamMarketAnalysis/blob/master/IPython%20Notebook/itemOut.csv " itemOut.csv")

## Disclaimer

All data examined has been freely provided by Valve Software via their APIs for the purpose of allowing users to examine this publicly available information.

[Powered by Steam](http://steampowered.com)